<a href="https://colab.research.google.com/github/cianc/AoC2023/blob/main/day13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -Uqq ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00


In [44]:
TEST = False #@param {type: "boolean"}
TEST_INPUT1 = 'day13_test1.txt'
TEST_INPUT2 = 'day13_test1.txt'
INPUT = 'day13_input1.txt'
PART=2 #@param {type: "integer"}

This is pretty disgusting, but it was fun doing it with bitmasks and binary operations.

In [ ]:
import ipdb
import itertools
import math
import re
import time


def get_input() -> list[str]:
  if PART == 1:
    input = TEST_INPUT1 if TEST else INPUT
  else:
    input = TEST_INPUT2 if TEST else INPUT

  with open(input, 'r') as f:
    rows = f.read().splitlines()
  return rows


def parse_patterns(input: tuple[str]) -> list[tuple[dict[int, list[int]], dict[int, list[int]]]]:
  partial_patterns = []
  full_patterns = []

  rows_to_fingerprints = {}
  row_index = 0
  pattern_index = 0
  for row in input:
    if row == '':
      pattern_index += 1
      partial_patterns.append(rows_to_fingerprints)
      rows_to_fingerprints = {}
      row_index = 0
    else:
      row_fingerprint = 0
      for col_index, cell in enumerate(row):
        if cell == '#':
          row_fingerprint = row_fingerprint | 1 << col_index
      rows_to_fingerprints[row_index] = row_fingerprint
      row_index += 1
  partial_patterns.append(rows_to_fingerprints)

  # Yes this is disgusting but it's what I get for having fun with bitmasks.
  for rows_to_fingerprints in partial_patterns:
    rows = rows_to_fingerprints.keys()
    row_width = int(math.ceil(max([math.log2(f) for f in rows_to_fingerprints.values()])))
    cols_to_fingerprints = {}
    for col_index in range(row_width):
      col_fingerprint = 0
      for row_index in rows:
        row_fingerprint = rows_to_fingerprints[row_index]
        fingerprint_fragment = (row_fingerprint >> col_index) & 1
        col_fingerprint = col_fingerprint | fingerprint_fragment << row_index
      cols_to_fingerprints[col_index] = col_fingerprint
    full_patterns.append((rows_to_fingerprints, cols_to_fingerprints))

  return full_patterns

def find_folds(x_to_fingerprints: list[tuple[dict[int, int], dict[int, int]]]) -> tuple[int, int]:
  answer = {}

  for index, fingerprint in x_to_fingerprints.items():
    if index == len(x_to_fingerprints.items()) - 1:
      continue
    xor = fingerprint ^ x_to_fingerprints[index+1]
    if xor == 0:
      indexes_before = range(0, index + 1)
      indexes_after = range(index + 1, len(x_to_fingerprints.keys()))
      indexes_count_before = len(indexes_before)
      indexes_count_after = len(indexes_after)
      match_count = 0
      almost_match_count = 0
      for index_before, index_after in zip(indexes_before[::-1], indexes_after):
        before_after_xor = x_to_fingerprints[index_before] ^ x_to_fingerprints[index_after]
        if before_after_xor == 0:
          match_count += 1
        elif math.log2(before_after_xor).is_integer():
          almost_match_count += 1
      if match_count in (indexes_count_before, indexes_count_after):
        answer['fold'] = (index, index+1)
      elif almost_match_count == 1 and match_count in (indexes_count_before - 1, indexes_count_after - 1):
        answer['too_few_matches_fold'] = (index, index+1)
    # Check if adjacent rows/cols are off by just 1 bit.
    elif math.log2(xor).is_integer():
      indexes_before = range(0, index)
      indexes_after = range (index + 2, len(x_to_fingerprints.keys()))
      indexes_count_before = len(indexes_before)
      indexes_count_after = len(indexes_after)
      match_count = 0
      for index_before, index_after in zip(indexes_before[::-1], indexes_after):
        if x_to_fingerprints[index_before] == x_to_fingerprints[index_after]:
          match_count += 1
      if match_count in (indexes_count_before, indexes_count_after):
        answer['off_by_one_fold'] = (index, index+1)

  return answer


start_time = time.time()
input = get_input()
patterns = parse_patterns(input)
answer = 0

for i, (rows_to_fingerprints, cols_to_fingerprints) in enumerate(patterns):
  folds = find_folds(rows_to_fingerprints)
  if PART == 1:
    row_fold = folds.get('fold', None)
    if row_fold:
      answer += ((row_fold[0] + 1) * 100)

  elif PART == 2:
    row_too_few_matches_fold = folds.get('too_few_matches_fold', None)
    row_off_by_one_fold = folds.get('off_by_one_fold', None)
    if row_too_few_matches_fold:
      answer += ((row_too_few_matches_fold[0] + 1) * 100)
    elif row_off_by_one_fold:
      answer += ((row_off_by_one_fold[0] + 1) * 100)


  folds = find_folds(cols_to_fingerprints)
  if PART == 1:
    col_fold = folds.get('fold', None)
    if col_fold:
      answer += (col_fold[0] + 1)
  elif PART == 2:
    col_too_few_matches_fold = folds.get('too_few_matches_fold', None)
    col_off_by_one_fold = folds.get('off_by_one_fold', None)
    if col_too_few_matches_fold:
      answer += (col_too_few_matches_fold[0] + 1)
    elif col_off_by_one_fold:
      answer += (col_off_by_one_fold[0] + 1)

  print(f'answer ({i}): {answer}')
  print('=' * 40)
print(f'answer: {answer}')
print(f'run time: {time.time() - start_time} (s)')